## Gathering Data for Project, Part 2/2
#### Data to be collected:
- avg star rating of starting players - aka team's recruiting rank from 3 years prior or a similar metric
- FG data (makes 40+, 50+)
- Red zone offensive efficiency, trips, scores, points per trip
- opponent TD percentage (defensive)
- starting field position of drives and avg starting field position
- Any other interesting advanced metrics

This can be iterated on in the future, but just trying to get a good starting point right now

Useful website for ideas and code stealing: https://blog.collegefootballdata.com/

Website for object documentation: https://github.com/CFBD/cfbd-python/tree/master/docs

In [4]:
# Not sure if all these imports are necessary, but it doesn't take too long so here they are
import requests
import pandas as pd
import numpy as np
from numpy import random
from time import sleep
import cfbd

### Setting up API config

In [5]:
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = 'gufskX77Qa6wsIhBQryuZYKZ4djpebOqvpj5Q/Rc2nyE7t6DQYNqN14ziEYpKa5q'
# original key: cCUuy68SmBD3qQ/HK1HZ0cks/gHrSkPiSUdcom44hDyV49P3syOpoiFJnGq8MDDY
# backup key: gufskX77Qa6wsIhBQryuZYKZ4djpebOqvpj5Q/Rc2nyE7t6DQYNqN14ziEYpKa5q

configuration.api_key_prefix['Authorization'] = 'Bearer'

api_config = cfbd.ApiClient(configuration)

In [6]:
games_api = cfbd.GamesApi(api_config)
drives_api = cfbd.DrivesApi(api_config)

### Player Game Stats, which will give us the kicking length stats per game

In [4]:
player_game_list = []



# loop over all the weeks in a season, up to 15 possible
for year in range(2014,2024):
    for week in range(1, 16):
    # make the api call to gather all the game stats for a given week
        games_week = games_api.get_player_game_stats(year=year, week=week, category = 'kicking')
        
        if len(games_week) == 0:
            continue
        
        player_game_list.append(games_week)
            
        print(f"year {year} week {week} finished")
        sleep(.5)

year 2014 week 1 finished
year 2014 week 2 finished
year 2014 week 3 finished
year 2014 week 4 finished
year 2014 week 5 finished
year 2014 week 6 finished
year 2014 week 7 finished
year 2014 week 8 finished
year 2014 week 9 finished
year 2014 week 10 finished
year 2014 week 11 finished
year 2014 week 12 finished
year 2014 week 13 finished
year 2014 week 14 finished
year 2014 week 15 finished
year 2015 week 1 finished
year 2015 week 2 finished
year 2015 week 3 finished
year 2015 week 4 finished
year 2015 week 5 finished
year 2015 week 6 finished
year 2015 week 7 finished
year 2015 week 8 finished
year 2015 week 9 finished
year 2015 week 10 finished
year 2015 week 11 finished
year 2015 week 12 finished
year 2015 week 13 finished
year 2015 week 14 finished
year 2015 week 15 finished
year 2016 week 1 finished
year 2016 week 2 finished
year 2016 week 3 finished
year 2016 week 4 finished
year 2016 week 5 finished
year 2016 week 6 finished
year 2016 week 7 finished
year 2016 week 8 finished


#### Iterating through the player_game_list and extracting only the data points we care about
I created a few placeholder lists/arrays to hold information, since iterating through the lists like list[0][0] didn't work
Think this has something to do with the object classes within player_game_data... But not totally sure

In [114]:
# Name columns
column_name_list = ['game_id','school','home_away','stat_type_1','stat_value_1','stat_type_2','stat_value_2']

# Empty array with width 7, length is # of games in data + 1 for the column names
games_played = 1
# 19K some odd rows will be needed over the 
for a in range(0,len(player_game_list)):
    for x in range(0,len(player_game_list[a])):
        for y in range (0,len(player_game_list[a][x].teams)):
            games_played = games_played + 1

placeholder_array = np.empty(((games_played + 1), 7), dtype=object)

# put columns into array
placeholder_array[0] = column_name_list

In [115]:

counter = 0

# This is a disgusting block of code, but it works and it took me way too long so I don't care
for a in range(0,len(player_game_list)):
    for x in range(0,len(player_game_list[a])):
        for y in range (0,len(player_game_list[a][x].teams)):
            
            # holding list which will reset every iteration
            holding_list = []

            # gets game ID
            holding_list.append(player_game_list[a][x].id)

            # gets school name and home/away value
            holding_list.append(player_game_list[a][x].teams[y].school)
            holding_list.append(player_game_list[a][x].teams[y].home_away)
            
            for z in range(0,len(player_game_list[a][x].teams[y].categories)):
                for items in player_game_list[a][x].teams[y].categories[z].types:
                    if player_game_list[a][x].teams[y].categories[z].name == 'kicking':
                    
                        # Longest made FG that game
                        if items.name == 'LONG':
                            holding_list.append(items.name)
                            holding_list.append(items.athletes[0].stat)
                        
                        # FG Makes over Attempts
                        if items.name == 'FG':
                            holding_list.append(items.name)
                            holding_list.append(items.athletes[0].stat)

            counter = counter + 1

            #Had to add this where values were not present
            if len(holding_list) == 7:
                placeholder_array[counter] = holding_list


# print(holding_list)
# [i for i,x in enumerate(error_list) if x==5] # => [1, 3]# print(placeholder_array)

#### Final results from the placeholder_array go in a DataFrame

In [118]:
df = pd.DataFrame.from_records(placeholder_array)
df.columns = df.iloc[0]
df = df.drop([0])
df.head()

,game_id,school,home_away,stat_type_1,stat_value_1,stat_type_2,stat_value_2
1,400547982,Ohio State,away,FG,2/2,LONG,46
2,400547982,Navy,home,FG,1/2,LONG,32
3,400610199,San Diego State,home,FG,3/4,LONG,43
4,400610199,Navy,away,FG,1/1,LONG,24
5,400548010,Troy,away,FG,1/2,LONG,47


Save FG Length stats to CSV

In [85]:
df.drop_duplicates(inplace=True)
df.to_csv('all_games_fg_length_data.csv')

If you don't want to run all that code and already have the csv, then read it in here

In [7]:
df = pd.DataFrame(data = pd.read_csv('all_games_fg_length_data.csv'))
df.head()

,Unnamed: 0,game_id,school,home_away,stat_type_1,stat_value_1,stat_type_2,stat_value_2
0,1,400547982.0,Ohio State,away,FG,2/2,LONG,46
1,2,400547982.0,Navy,home,FG,1/2,LONG,32
2,3,400610199.0,San Diego State,home,FG,3/4,LONG,43
3,4,400610199.0,Navy,away,FG,1/1,LONG,24
4,5,400548010.0,Troy,away,FG,1/2,LONG,47


Accessing Box Score from individual games so that we can get avg. field position per game

Only accesses game ID, so we'll need to reference df

In [8]:
# Starting Lists
# grab game ids from df
dupe_ids_dirty = []
dupe_ids_dirty = df.game_id.tolist()


ids = []

# Deduping IDs (OG table has 2 IDs per row)
dupe_ids = [x for x in dupe_ids_dirty if not np.isnan(x)]
[ids.append(int(x)) for x in dupe_ids if x not in ids]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

Get field position data from server

In [12]:
teams_list = []
avg_field_position_list =[]
id_list = []

count = 0
for x in ids:
        # some of the IDs were giving errors and quitting out of the whole function
        # had to do a try and except in order to keep the calls going despite the failures
        try:
                box_score = games_api.get_advanced_box_score(x)
                id_list.append(x)
                id_list.append(x)
                # have to do this twice so it matches the number of records in box_score_teams_field_position
                for i in box_score.teams.field_position:
                        teams_list.append(i.team)
                        avg_field_position_list.append(i.average_start)
        except:
                print(str(x) + ' failed the API call')
                continue
        finally:
                count = count + 1
                if count == 100:
                        break
                sleep(.1)

400548169 failed the API call
400548173 failed the API call


In [44]:
# These numbers should match up to fit into the dataframe nicely
# print(len(id_list))
# print(len(teams_list))
# print(len(avg_field_position_list))

# prepping the data in a single array
id_array = np.asarray(id_list, dtype = object)
id_array.transpose
teams_array = np.asarray(teams_list, dtype = object)
teams_array.transpose
pos_array = np.asarray(avg_field_position_list, dtype = object)
pos_array.transpose

big_array = np.stack((id_array,teams_array,pos_array), axis = 1)

array([400547982, 'Ohio State', 69.8], dtype=object)

In [46]:
pos_df = pd.DataFrame(data = big_array, columns = ['game_id','team','avg_field_position'])
pos_df.head()

,game_id,team,avg_field_position
0,400547982,Ohio State,69.8
1,400547982,Navy,78.6
2,400610199,San Diego State,57.6
3,400610199,Navy,69.9
4,400548010,Troy,71.3


In [47]:
pos_df.drop_duplicates(inplace=True)
pos_df.to_csv('all_games_field_pos_data.csv')

### Team Recruiting Rankings

In [55]:
# Getting the data

recruiting_api = cfbd.RecruitingApi(api_config)

team_recruiting_rank = []

for i in range(2014, 2024):
    recruiting_year = recruiting_api.get_recruiting_teams(year = i)
    team_recruiting_rank.append(recruiting_year)
    print(f"year {i} finished")
    sleep(.5)


year 2014 finished
year 2015 finished
year 2016 finished
year 2017 finished
year 2018 finished
year 2019 finished
year 2020 finished
year 2021 finished
year 2022 finished
year 2023 finished


In [63]:
team_recruiting_rank[5][1]

{'points': 309.12, 'rank': 2, 'team': 'Georgia', 'year': 2019}

In [78]:
yearly_recruit = []

yearly_recruit = [x for l in team_recruiting_rank for x in l]
print(yearly_recruit)


recruit_df = (pd.DataFrame.from_records([dict(recruit_pts = r.points, rank = r.rank, team = r.team, year = r.year) for r in yearly_recruit]))

recruit_df.head()
# [dict(home_team=g.home_team, away_team = g.away_team, venue = g.venue
#                                             ,home_score = g.home_points, away_score = g.away_points) for g in games_2023])


[{'points': 319.71, 'rank': 1, 'team': 'Alabama', 'year': 2014}, {'points': 299.29, 'rank': 2, 'team': 'LSU', 'year': 2014}, {'points': 296.08, 'rank': 3, 'team': 'Ohio State', 'year': 2014}, {'points': 286.79, 'rank': 4, 'team': 'Florida State', 'year': 2014}, {'points': 278.08, 'rank': 5, 'team': 'Texas A&M', 'year': 2014}, {'points': 277.12, 'rank': 6, 'team': 'Auburn', 'year': 2014}, {'points': 274.77, 'rank': 7, 'team': 'Tennessee', 'year': 2014}, {'points': 273.39, 'rank': 8, 'team': 'Georgia', 'year': 2014}, {'points': 267.4, 'rank': 9, 'team': 'Florida', 'year': 2014}, {'points': 264.86, 'rank': 10, 'team': 'USC', 'year': 2014}, {'points': 260.44, 'rank': 11, 'team': 'Notre Dame', 'year': 2014}, {'points': 256.08, 'rank': 12, 'team': 'Miami', 'year': 2014}, {'points': 250.05, 'rank': 13, 'team': 'Stanford', 'year': 2014}, {'points': 248.94, 'rank': 14, 'team': 'Oklahoma', 'year': 2014}, {'points': 241.12, 'rank': 15, 'team': 'Ole Miss', 'year': 2014}, {'points': 240.73, 'rank':

,recruit_pts,rank,team,year
0,319.71,1,Alabama,2014
1,299.29,2,LSU,2014
2,296.08,3,Ohio State,2014
3,286.79,4,Florida State,2014
4,278.08,5,Texas A&M,2014


Save to csv

In [79]:
recruit_df.drop_duplicates(inplace=True)
recruit_df.to_csv('all_teams_recruit_data.csv')